# ISA Jupyter Notebook

Provided by X-Omics.
Adjusted by Christel van Haren.
Last modifaction on 27/6/2023.


In [17]:
# Set working directory, all paths will be relative (downstream) to this directory

import os
working_directory = "/Users/christelvanharen/Documents/StageRadboud/isatools/TWOCdemonstrator/data/ISA_test"  



In [18]:
import os
os.listdir()

['isa-to-rdfs.py',
 'TWOC_create_isa original.ipynb',
 'TWOC_create_isa.ipynb',
 'README.md',
 '.ipynb_checkpoints',
 'TWOC_test_isa.ipynb']

In [19]:
# Read in Patient Metadata

import pandas as pd

patient_metadata = pd.read_csv('/Users/christelvanharen/Documents/StageRadboud/isatools/TWOCdemonstrator/data/ISA_test/metadata.tsv', sep='\t')
patient_metadata.columns = patient_metadata.columns.str.replace(' ', '_', regex=True)
patient_metadata


,SAMPLE-ID,FILENAME_fwd,FILENAME_rev,SUBJECT-ID,RUN,GENDER,AGE_(Y),HEIGHT_(CM),WEIGHT_(KG),BMI,DIETARY_SUPPLEMENTS,PROBIOTICS,DIET,DONATATION,SUFFIX,RANKSTAT_GENDER,DESCRIPTION
0,sample028,sample028_S001_L001_R1_001.fastq.gz,sample028_S001_L001_R2_001.fastq.gz,FT-10038,run_6,M,33,"180,34","80,38","24,73",Vitamin B12 and D,NaN,Vegetarian,AF,NaN,NaN,NaN
1,sample031,sample031_S001_L001_R1_001.fastq.gz,sample031_S001_L001_R2_001.fastq.gz,FT-10038,run_6,M,33,"180,34","80,38","24,73",Vitamin B12 and D,NaN,Vegetarian,AS,NaN,NaN,NaN
2,sample118,sample118_S001_L001_R1_001.fastq.gz,sample118_S001_L001_R2_001.fastq.gz,FT-10038,run_10,M,33,"182,9","72,35","24,73",Vitamin B12 and D,NaN,Vegetarian,BI,NaN,NaN,NaN
3,sample121,sample121_S001_L001_R1_001.fastq.gz,sample121_S001_L001_R2_001.fastq.gz,FT-10042,run_10,M,23,"182,9","72,35","21,51",NaN,Yakult probiotic drink,NaN,BH,NaN,NaN,NaN
4,sample126,sample126_S001_L001_R1_001.fastq.gz,sample126_S001_L001_R2_001.fastq.gz,FT-10048,run_10,M,36,"182,9","72,35","22,2",NaN,NaN,NaN,AM,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,sample184,sample184_S001_L001_R1_001.fastq.gz,sample184_S001_L001_R2_001.fastq.gz,FT-10911,run_11,M,24,"177,8","87,14","21,22",NaN,NaN,NaN,BI,NaN,NaN,NaN
119,sample187,sample187_S001_L001_R1_001.fastq.gz,sample187_S001_L001_R2_001.fastq.gz,FT-10959,run_11,M,35,"188,21","79,15","17,08",NaN,NaN,NaN,BI,NaN,NaN,NaN
120,sample188,sample188_S001_L001_R1_001.fastq.gz,sample188_S001_L001_R2_001.fastq.gz,FT-10959,run_11,M,35,"188,21","79,15","17,08",NaN,NaN,NaN,AI,NaN,NaN,NaN
121,sample189,sample189_S001_L001_R1_001.fastq.gz,sample189_S001_L001_R2_001.fastq.gz,FT-11094,run_11,F,19,"173,36","90,4","26,37",NaN,NaN,NaN,BI,NaN,NaN,NaN


In [20]:
# Set Sample IDs to strings (some control samples are integers)
patient_metadata['SAMPLE-ID'] = patient_metadata['SAMPLE-ID'].astype(str)

In [21]:
# All the ontologies sources in a library

from isatools.model import *

ontologies = {
    "COVOC": OntologySource(
        name = "CoVoc Coronavirus Vocabulary",
        file = "http://purl.obolibrary.org/obo/covoc.owl",
        description = "The COVID-19 Vocabulary (COVoc) is an ontology containing terms related to the research of the COVID-19 pandemic. This includes host organisms, pathogenicity, gene and gene products, barrier gestures, treatments and more."),  
    "CRO": OntologySource(
        name = "CRO - Contributor Role Ontology",
        file = "http://purl.obolibrary.org/obo/cro.owl",
        description = "A classification of the diverse roles performed in the work leading to a published research output in the sciences. Its purpose to provide transparency in contributions to scholarly published work, to enable improved systems of attribution, credit, and accountability."),
    "EDAM": OntologySource(
        name = "EDAM - EMBRACE Data and Methods",
        file = "http://edamontology.org/EDAM.owl",
        description = "Bioinformatics operations, data types, formats, identifiers and topics"),  
    "EFO": OntologySource(
        name = "EFO - Experimental Factor Ontology", 
        file = "http://www.ebi.ac.uk/efo/efo.owl",
        description = "The Experimental Factor Ontology (EFO) provides a systematic description of many experimental variables available in EBI databases, and for external projects such as the NHGRI GWAS catalogue. It combines parts of several biological ontologies, such as anatomy, disease and chemical compounds. The scope of EFO is to support the annotation, analysis and visualization of data handled by many groups at the EBI and as the core ontology for OpenTargets.org"), 
    "GECKO": OntologySource(
        name = "GECKO - Genomics Cohorts Knowledge Ontology",
        file = "http://purl.obolibrary.org/obo/gecko.owl",
        description = "An ontology to represent genomics cohort attributes."),
    "NCBITAXON": OntologySource(
        name = "NCBI organismal classification", 
        file = "http://purl.obolibrary.org/obo/ncbitaxon.owl",
        description = "An ontology representation of the NCBI organismal taxonomy"),
    "NCIT": OntologySource(
        name = "NCI Thesaurus OBO Edition", 
        file = "http://purl.obolibrary.org/obo/ncit.owl",
        description = "The NCIt OBO Edition project aims to increase integration of the NCIt with OBO Library ontologies. NCIt is a reference terminology that includes broad coverage of the cancer domain, including cancer related diseases, findings and abnormalities. NCIt OBO Edition releases should be considered experimental."),
    "NGBO": OntologySource(
        name = "Next generation biobanking ontology (NGBO).", 
        file = "http://purl.obolibrary.org/obo/obi.owl",
        description = "Next Generation Biobanking Ontology (NGBO) is an open application ontology representing contextual data about omics digital assets in biobank. The ontology focuses on capturing the information about three main activities: wet bench analysis used to generate omics data, bioinformatics analysis used to analyze and interpret data, and data management."),
    "OBI": OntologySource(
        name = "OBI - Ontology for Biomedical Investigations", 
        file = "http://purl.obolibrary.org/obo/ngbo.owl ",
        description = "An integrated ontology for the description of life-science and clinical investigations"),
    "PRIDE": OntologySource(
        name = "PRIDE Controlled Vocabulary",
        file = "http://purl.obolibrary.org/obo/pride_cv.obo",
        description = "The PRIDE PRoteomics IDEntifications (PRIDE) database is a centralized, standards compliant, public data repository for proteomics data, including protein and peptide identifications, post-translational modifications and supporting spectral evidence."),
    "SO": OntologySource(
        name = "Sequence types and features ontology", 
        file = "http://purl.obolibrary.org/obo/so.owl ",
        description = "A structured controlled vocabulary for sequence annotation, for the exchange of annotation data and for the description of sequence objects in databases."),
    "STATO": OntologySource(
        name = "STATO: the statistical methods ontology", 
        file = "http://purl.obolibrary.org/obo/stato.owl ",
        description = "STATO is the statistical methods ontology. It contains concepts and properties related to statistical methods, probability distributions and other concepts related to statistical analysis, including relationships to study designs and plots. "),
    "UBERON": OntologySource(
        name = "Uber-anatomy ontology",
        file = "http://purl.obolibrary.org/obo/uberon.owl",
        description = "Uberon is an integrated cross-species anatomy ontology representing a variety of entities classified according to traditional anatomical criteria such as structure, function and developmental lineage. The ontology includes comprehensive relationships to taxon-specific anatomical ontologies, allowing integration of functional, phenotype and expression data.")
}


In [22]:
# Filling the first ISA part, the Investigation, with the publication and the main aut

investigation = Investigation(
    filename = "i_investigation.txt", 
    identifier = "https://doi.org/10.1186/s13104-021-05520-z", 
    title = "16S rRNA sequencing of samples from universal stool bank donors.",
    description = "Universal stool banks provide stool to physicians for use in treating recurrent Clostridioides difficile infection via fecal microbiota transplantation. Stool donors providing the material are rigorously screened for diseases and disorders with a potential microbiome etiology, and they are likely healthier than the controls in most microbiome datasets. 16S rRNA sequencing was performed on samples from a selection of stool donors at a large stool bank, OpenBiome, to characterize their gut microbial community and to compare samples across different timepoints and sequencing runs.",
    submission_date = "",
    public_release_date = "2021-03-23",
    ontology_source_references = [o for o in ontologies.values()],
     publications = [
        Publication(doi="https://doi.org/10.1186/s13104-021-05520-z", 
                    title='16S rRNA sequencing of samples from universal stool bank donors.',
                    status=OntologyAnnotation(
                                term="preprint",
                                term_source= ontologies["EFO"],
                                term_accession="http://www.ebi.ac.uk/efo/EFO_0010558"),
                    author_list="Marina Santiago, Scott W. Olesen")],
    
    contacts = [
        Person(
            last_name = "Olesen", 
            first_name = "Scott",
            mid_initials = "W.",
            affiliation = "OpenBiome, 2067 Massachusetts Ave, Cambridge, MA, 02140, USA",
            email = "solesen@openbiome.org",
            address = "https://openbiome.org",
            roles = [
                OntologyAnnotation(
                    term = "project management role",
                    term_source = ontologies["CRO"], 
                    term_accession ="http://purl.obolibrary.org/obo/CRO_0000065")])],
    studies = None,
    comments = None)

In [23]:
# Filling in the second part of the ISA, the Study, a more detailed version of the Investigation

cohort_study = Study(
    filename = "s_study.txt", 
    identifier = "https://doi.org/10.1186/s13104-021-05520-z", 
    title = "16S rRNA sequencing of samples from universal stool bank donors.",
    description = "Universal stool banks provide stool to physicians for use in treating recurrent Clostridioides difficile infection via fecal microbiota transplantation. Stool donors providing the material are rigorously screened for diseases and disorders with a potential microbiome etiology, and they are likely healthier than the controls in most microbiome datasets. 16S rRNA sequencing was performed on samples from a selection of stool donors at a large stool bank, OpenBiome, to characterize their gut microbial community and to compare samples across different timepoints and sequencing runs.", 
    submission_date = "", 
    public_release_date = "2021-03-23",
    contacts = [
        Person(
            last_name = "Olesen", 
            first_name = "Scott",
            mid_initials = "W.",
            affiliation = "OpenBiome, 2067 Massachusetts Ave, Cambridge, MA, 02140, USA",
            roles = [
                OntologyAnnotation(
                    term = "author role",
                    term_source = ontologies["CRO"], 
                    term_accession ="http://purl.obolibrary.org/obo/CRO_0000001")]),
        Person(
            last_name = "Santiago", 
            first_name = "Marina",
            # mid_initials = "",
            affiliation = "OpenBiome, 2067 Massachusetts Ave, Cambridge, MA, 02140, USA",
            roles = [
                OntologyAnnotation(
                    term = "author role",
                    term_source = ontologies["CRO"], 
                    term_accession ="http://purl.obolibrary.org/obo/CRO_0000001")])],
    

    design_descriptors = [
        OntologyAnnotation(
                term = "Multi-omics study",
                term_source = ontologies["PRIDE"],
                term_accession = "http://purl.obolibrary.org/obo/PRIDE_0000461"),
        ],
    
    publications = [
        Publication(doi="https://doi.org/10.1186/s13104-021-05520-z", pubmed_id= '33757553',
                    title='16S rRNA sequencing of samples from universal stool bank donors',
                    status=OntologyAnnotation(term="indexed in PubMed"),
                    author_list="Marina Santiago, Scott W. Olesen")],
    factors = None, 
    protocols = None,
    assays = None,
    sources = None,
    samples = None,
    process_sequence = None,
    other_material = None,
    characteristic_categories = None,
    comments = None,
    units = None)
investigation.studies.append(cohort_study)

In [24]:
# Define protocol parameters, used in protocols

protocol_params = {
    "Metadata": ProtocolParameter(
        parameter_name = OntologyAnnotation(
            term = "Metadata",
            term_source = ontologies["NCIT"],
            term_accession = "http://purl.obolibrary.org/obo/NCIT_C52095")
        ),

    "16s RNA sequence data": ProtocolParameter(
        parameter_name = OntologyAnnotation(
            term = "16s RNA sequence data",
            term_source = ontologies["NGBO"],
            term_accession = "http://purl.obolibrary.org/obo/NGBO_6000204 ")
        ),

    "low-input metagenomic next-generation sequencing": ProtocolParameter(
        parameter_name = OntologyAnnotation(
            term = "low-input metagenomic next-generation sequencing",
            term_source = ontologies["COVOC"],
            term_accession = "http://purl.obolibrary.org/obo/COVOC_0010015")
        ),

    "adapter-sequence trimming": ProtocolParameter(
        parameter_name = OntologyAnnotation(
            term = "adapter-sequence trimming",
            term_source = ontologies["OBI"],
            term_accession = "http://purl.obolibrary.org/obo/OBI_0002565 ")
        ),

    "PCR primers": ProtocolParameter(
        parameter_name = OntologyAnnotation(
            term = "PCR primers",
            term_source = ontologies["EDAM"],
            term_accession = "http://edamontology.org/data_1240 ")
        ),

}

In [25]:
# Define Assays

assays = {
    "metataxonomics": Assay(
        filename = "a_assay_metataxonomics.txt",
        measurement_type = OntologyAnnotation(
            term = "microbiome sequencing assay",
            term_source = ontologies["GECKO"],
            term_accession = "http://purl.obolibrary.org/obo/GECKO_0000040"),

        technology_type = OntologyAnnotation(
            term = "microbiome sequencing assay",
            term_source = ontologies["GECKO"],
            term_accession = "http://purl.obolibrary.org/obo/GECKO_0000040")
        )
}

In [26]:
# Define protocols

protocols = {
    
    "sample_collection": Protocol( 
        name = "material sampling process",
        protocol_type = OntologyAnnotation(
            term = "material sampling process",
            term_source = ontologies["OBI"],  
            term_accession = "http://purl.obolibrary.org/obo/OBI_0000744"),
        description = "A specimen gathering process with the objective to obtain a specimen that is representative of the input material entity",
        parameters =[protocol_params["Metadata"], protocol_params["16s RNA sequence data"], protocol_params["low-input metagenomic next-generation sequencing"]]
        ),
        
    "Data transformation": Protocol(
        name = "Data transformation",
        protocol_type = OntologyAnnotation(
            term = "data transformation",
            term_source = ontologies["OBI"],
            term_accession = "http://purl.obolibrary.org/obo/OBI_0200000"),
        description = "A planned process that produces output data from input data."
        ),

    "Feature annotation": Protocol(
        name = "feature annotation",
        protocol_type = OntologyAnnotation(
            term = "Annotation",
            term_source = ontologies["EDAM"],
            term_accession = "http://edamontology.org/operation_0226")
        ),    

    
    "Polymerase Chain Reaction": Protocol(
        name = "Polymerase Chain Reaction",
        protocol_type = OntologyAnnotation(
            term = "Polymerase Chain Reaction",
            term_source = ontologies["NCIT"],
            term_accession = "http://purl.obolibrary.org/obo/NCIT_C17003"),
        parameters = [protocol_params["PCR primers"]]
        ),
       
    "operational taxonomic unit matrix": Protocol( 
        name = "operational taxonomic unit matrix",
        protocol_type = OntologyAnnotation(
            term = "operational taxonomic unit matrix",
            term_source = ontologies["OBI"],  
            term_accession = "http://purl.obolibrary.org/obo/OBI_0001968"),
        description = "Operational Taxonomic Unit matrix is a data item, organized as a table, where organismal taxonomic units, computed by sequence analysis and genetic distance calculation, are counted in a set of biological or environmental samples. The table is used to appraise biodiversity of a population or community of living organism.",
        ),

    "demultiplexed sequence data": Protocol( 
        name = "demultiplexed sequence data",
        protocol_type = OntologyAnnotation(
            term = "demultiplexed sequence data",
            term_source = ontologies["OBI"],  
            term_accession = "http://purl.obolibrary.org/obo/OBI_0002601"),
        description = "Sequence data in which an identifier subsequence has been used to categorize each reads by source.",
        ),

    "Quality Control": Protocol( 
        name = "Quality Control",
        protocol_type = OntologyAnnotation(
            term = "Quality Control",
            term_source = ontologies["NCIT"],  
            term_accession = "http://purl.obolibrary.org/obo/NCIT_C15311 "),
        description = "Set of measurements and inspections taken to verify that performance of equipment and procedures remains within specified limits.",
        ),

}

# Append to study protocols
for protocol in protocols.values():
    cohort_study.protocols.append(protocol)

In [27]:
# The Characteristics are the headers of the metadata and are linked to it as well.

# Add samples
for index, row in patient_metadata.iterrows():
    
    # Create source
    source_name = row["SUBJECT-ID"]
    source = Source(
        name = source_name,
        characteristics = [
            Characteristic(
                category = OntologyAnnotation(
                    term = "Organism",
                    term_source = ontologies["OBI"],
                    term_accession = "http://purl.obolibrary.org/obo/OBI_0100026"),
                value = OntologyAnnotation(
                    term = "Homo sapiens",
                    term_source = ontologies["NCBITAXON"],
                    term_accession = "http://purl.obolibrary.org/obo/NCBITaxon_9606"))])
     
    
    # Iterate over phenotype information (and add if available)
    for meta_index, meta_row in patient_metadata.iterrows():
        if source_name == meta_row["SUBJECT-ID"]:
            source.characteristics.append(
                Characteristic(
                    category = OntologyAnnotation(
                        term = "Subject Identifier",
                        term_source = ontologies['NCIT'],
                        term_accession = "http://purl.obolibrary.org/obo/NCIT_C83083"),
                    value = meta_row["SUBJECT-ID"]))
            
            source.characteristics.append(
                Characteristic(
                    category = OntologyAnnotation(
                        term = "Sex",
                        term_source = ontologies['NCIT'],
                        term_accession = "http://purl.obolibrary.org/obo/NCIT_C28421"),
                    value = meta_row["GENDER"]))
            
            source.characteristics.append(
                Characteristic(
                    category = OntologyAnnotation(
                        term = "Age-Years",
                        term_source = ontologies['NCIT'],
                        term_accession = "http://purl.obolibrary.org/obo/NCIT_C37908"),
                    value = meta_row["AGE_(Y)"]))
        
            source.characteristics.append(
                Characteristic(
                    category = OntologyAnnotation(
                        term = "Body Mass Index)",
                        term_source = ontologies['NCIT'],
                        term_accession = "http://purl.obolibrary.org/obo/NCIT_C16358"),
                    value = meta_row["BMI"]))
            
            source.characteristics.append(
                Characteristic(
                    category = OntologyAnnotation(
                        term = "Body Height",
                        term_source = ontologies['NCIT'],
                        term_accession = "http://purl.obolibrary.org/obo/NCIT_C164634"),
                    value = meta_row["HEIGHT_(CM)"]))
            
            source.characteristics.append(
                Characteristic(
                    category = OntologyAnnotation(
                        term = "Body Weight",
                        term_source = ontologies['NCIT'],
                        term_accession = "http://purl.obolibrary.org/obo/NCIT_C81328"),
                    value = meta_row["WEIGHT_(KG)"]))
            
            source.characteristics.append(
                Characteristic(
                    category = OntologyAnnotation(
                        term = "Dietary Supplement",
                        term_source = ontologies['NCIT'],
                        term_accession = "http://purl.obolibrary.org/obo/NCIT_C1505"),
                    value = meta_row["DIETARY_SUPPLEMENTS"]))
            
            source.characteristics.append(
                Characteristic(
                    category = OntologyAnnotation(
                        term = "Probiotics",
                        term_source = ontologies['NCIT'],
                        term_accession = "http://purl.obolibrary.org/obo/NCIT_C93144"),
                    value = meta_row["PROBIOTICS"]))
            
            source.characteristics.append(
                Characteristic(
                    category = OntologyAnnotation(
                        term = "Diet",
                        term_source = ontologies['NCIT'],
                        term_accession = "http://purl.obolibrary.org/obo/NCIT_C15222 "),
                    value = meta_row["DIET"]))
    
    # Create sample
    sample_name = row["SAMPLE-ID"]
    stool_sample = Sample(
        name = sample_name, 
        derives_from = [source])
    stool_sample.characteristics.append(
        Characteristic(
            category = OntologyAnnotation(
                term = "anatomical entity",
                term_source = ontologies["UBERON"],
                term_accession = "http://purl.obolibrary.org/obo/UBERON_0001062"),
                ))
    cohort_study.samples.append(stool_sample)
    
    # Sample collection_process                
    sample_collection_process = Process(
        name = "samplecollection_{0}".format(row["SUBJECT-ID"]),
        executes_protocol = protocols["sample_collection"],
        inputs = [source],
        outputs = [stool_sample])
    cohort_study.process_sequence.append(sample_collection_process)

In [28]:
# Add samples to metataxonomics assay

# Metataxonomics subdirectory
metataxonomics_dir = os.path.join(working_directory)

# Datafiles 
raw_datafile = DataFile(filename= os.path.join("feature-matrices-general.csv"), label = "Feature Matrices General")
feature_annotation_file = DataFile(filename = os.path.join("metataxonomics-fmeta.csv"), label = "Metataxonomics Feature Metadata")


# Iterate over samples in study
for index, row in patient_metadata.iterrows():
    for idx, sample in enumerate(cohort_study.samples):
        # Linking the samples to the header of the metadata file
        if sample.name == row["SAMPLE-ID"]:

            # Samples
            material_extract = Material(
                        name = "extract_{0}".format(sample.name),
                        type_ = "Extract Name",
                    )
            
            sequencing_extract = ParameterValue(
                        category = protocol_params["16s RNA sequence data"],
                        value = "16s RNA sequence data"
            )

            metadata = ParameterValue(
                        category = protocol_params["Metadata"], 
                        value = ""
                    )

            extraction_process = Process(
                        executes_protocol = protocols["sample_collection"],
                        parameter_values=[sequencing_extract, metadata],
                        inputs = [sample],
                        outputs = [material_extract]
                    )

            # Data transformation
            data_transformation_process = Process(
                        name = "data_transformation_{0}".format(sample.name),
                        executes_protocol = protocols["Data transformation"],
                        inputs = [material_extract],
                        outputs = [raw_datafile]  
                    )

            # Feature annotation
            feature_annotation_process = Process(
                        name = "feature_annotation_{0}".format(material_extract.name),
                        executes_protocol = protocols["Feature annotation"],
                        inputs = [raw_datafile],
                        outputs = [feature_annotation_file]
            )

            
            # Link processes, is for linking the processes to each other
            plink(extraction_process, data_transformation_process)
            plink(data_transformation_process, feature_annotation_process)


            # Add samples, materials and data files to the assay
            assays["metataxonomics"].samples.append(stool_sample)
            assays["metataxonomics"].other_material.append(material_extract)
            assays["metataxonomics"].data_files.append(raw_datafile)
            assays["metataxonomics"].data_files.append(feature_annotation_file)


            ## Add processes to the assay
            assays["metataxonomics"].process_sequence.append(extraction_process)
            assays["metataxonomics"].process_sequence.append(data_transformation_process)
            assays["metataxonomics"].process_sequence.append(feature_annotation_process)


In [29]:
# Filling in the third and last part of the ISA, the Assay

# Add assays to cohort study
for assay in assays.values():
        cohort_study.assays.append(assay)

## Write ISA-Tab files

In [30]:
# Write to ISA-Tab
from isatools import isatab
isatab.dump(investigation, working_directory)
print()

2023-06-02 13:36:41,634 [INFO]: graph.py(_all_end_to_end_paths:20) >> [863, 866, 869, 872, 875, 878, 881, 884, 887, 890, 893, 896, 899, 902, 905, 908, 911, 914, 917, 920, 923, 926, 929, 932, 935, 938, 941, 944, 947, 950, 953, 956, 959, 962, 965, 968, 971, 974, 977, 980, 983, 986, 989, 992, 995, 998, 1001, 1004, 1007, 1010, 1013, 1016, 1019, 1022, 1025, 1028, 1031, 1034, 1037, 1040, 1043, 1046, 1049, 1052, 1055, 1058, 1061, 1064, 1067, 1070, 1073, 1076, 1079, 1082, 1085, 1088, 1091, 1094, 1097, 1100, 1103, 1106, 1109, 1112, 1115, 1118, 1121, 1124, 1127, 1130, 1133, 1136, 1139, 1142, 1145, 1148, 1151, 1154, 1157, 1160, 1163, 1166, 1169, 1172, 1175, 1178, 1181, 1184, 1187, 1190, 1193, 1196, 1199, 1202, 1205, 1208, 1211, 1214, 1217, 1220, 1223, 1226, 1229]
2023-06-02 13:36:41,636 [WARNING]: write.py(write_study_table_files:62) >> [865, 864, 863, 868, 867, 866, 871, 870, 869, 874, 873, 872, 877, 876, 875, 880, 879, 878, 883, 882, 881, 886, 885, 884, 889, 888, 887, 892, 891, 890, 895, 894, 8

## Write to ISA-json

In [31]:
# Write the ISA-Tab file into a JSON file
import json
from isatools.isajson import ISAJSONEncoder
with open(os.path.join(working_directory, "isa.json"), "w") as out_file:
    json.dump(
        investigation, 
        out_file,
        cls = ISAJSONEncoder, 
        sort_keys = True, 
        indent = 4, 
        separators = (',', ': '))